In [36]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Javascript, display, HTML, Markdown as md
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import json
import requests
import pandas as pd

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [37]:
banks = {
        'DBS': {
                'ticker': 'D05',
                'morningstarID': '0P0000A67J'
        },
        'OCBC':  {
                'ticker': 'O39',
                'morningstarID': '0P0000A68H'
        },
        'UOB':{
                'ticker': 'U11',
                'morningstarID': '0P0000A692'
        }
}
print(banks.keys())

dict_keys(['DBS', 'OCBC', 'UOB'])


In [38]:
dropdown = widgets.Dropdown(options = sorted(banks.keys()))
chosen = 'DBS'

def dropdown_eventhandler(change):
    global chosen 
    chosen = change.new
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
    
dropdown.observe(dropdown_eventhandler, names='value')

display(dropdown)


Dropdown(options=('DBS', 'OCBC', 'UOB'), value='DBS')

In [39]:
finUrl = 'https://api-global.morningstar.com/sal-service/v1/stock/newfinancials/{id}/{report}/detail?dataType=A&reportType=A&locale=en&&operation=export&clientId=MDC&benchmarkId=category&version=3.21.1'
finHeaders = {
    'Referer': 'https://www.morningstar.com/', 
    'apikey': 'lstzFDEOhfFNMLikKa0am9mgEKLBl49T', 
    'x-sal-contenttype': 'nNsGdN3REOnPMlKDShOYjlk6VYiEVLSdpfpXAm7o2Tk='
}

keyRatioUrl = 'http://financials.morningstar.com/finan/ajax/exportKR2CSV.html?&callback=?&t=XSES:{ticker}&region=sgp&culture=en-US&cur=&order=asc'
keyRatioHeaders = {
    'Referer': 'http://financials.morningstar.com/ratios/r.html?t=D05&region=sgp&culture=en-US'
}

# Download finacial reports
reports = ['balancesheet', 'incomestatement', 'cashflow']
for report in reports:
    print('Downloading % s'%report)
    with open('% s.xls'%report, 'wb') as f:
        response = requests.get(finUrl.format(id = banks[chosen]['morningstarID'], report = report), headers=finHeaders)
        f.write(response.content)
        print('Done')


# Download key ratio data
with open('keyratios.csv', 'wb') as f:
    print('Downloading Key Ratios data')
    response = requests.get(keyRatioUrl.format(ticker = banks[chosen]['ticker']), headers=keyRatioHeaders)
    f.write(response.content)
    print('Done')

Done
Done
Done
Done


In [40]:
balance = pd.read_excel("balancesheet.xls",  index_col=0)
balance

,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010
Name,,,,,,,,,,
Total Assets,"578,946,000,000","550,751,000,000","517,711,000,000","481,570,000,000","457,834,000,000","440,666,000,000","402,008,000,000","353,033,000,000","340,847,000,000","283,710,000,000"
Cash and Cash Equivalents,"65,698,000,000","62,363,000,000","62,438,000,000","56,858,000,000","57,114,000,000","61,780,000,000","58,543,000,000","46,580,000,000","25,304,000,000","31,203,000,000"
Cash,"2,303,000,000","2,460,000,000","2,205,000,000","2,938,000,000","3,070,000,000","1,936,000,000","1,803,000,000","1,656,000,000",NaN,NaN
Due from Banks,"39,336,000,000","40,178,000,000","35,975,000,000","30,018,000,000","38,285,000,000","42,263,000,000","39,817,000,000","28,808,000,000",NaN,NaN
Deposits and Balances at Central Banks,"24,059,000,000","19,725,000,000","24,258,000,000","23,902,000,000","15,759,000,000","17,581,000,000","16,923,000,000","16,116,000,000",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
Non-Controlling/Minority Interests,"818,000,000","830,000,000","2,344,000,000","2,361,000,000","2,422,000,000","2,498,000,000","3,453,000,000","4,261,000,000","4,275,000,000","6,503,000,000"
Common Shares Issued,"2,554,121,000","2,551,615,000","2,557,067,485","2,536,110,000","2,499,907,000","2,472,363,866","2,442,997,000","2,464,391,695","2,450,000,000","2,408,683,976"
Common Shares Outstanding,"2,554,121,000","2,551,615,000","2,557,067,485","2,536,110,000","2,499,907,000","2,472,363,866","2,442,997,000","2,464,391,695","2,450,000,000","2,408,683,976"


In [41]:
income = pd.read_excel("incomestatement.xls",  index_col=0)
income

,TTM,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010
Name,,,,,,,,,,,
Total Revenue,"14,543,000,000","14,543,000,000","13,092,000,000","12,273,000,000","11,435,000,000","10,833,000,000","9,773,000,000","9,054,000,000","8,465,000,000","7,631,000,000","7,066,000,000"
Net Interest Income,"9,625,000,000","9,625,000,000","8,955,000,000","7,791,000,000","7,305,000,000","7,100,000,000","6,321,000,000","5,569,000,000","5,285,000,000","4,825,000,000","4,318,000,000"
"Interest Income, Revenue","15,592,000,000","15,592,000,000","13,798,000,000","10,833,000,000","9,748,000,000","9,644,000,000","8,948,000,000","7,986,000,000","7,621,000,000","6,555,000,000","5,699,000,000"
Interest Income from Loans and Lease,"10,247,000,000","10,247,000,000","8,959,000,000","7,096,000,000","6,628,000,000","6,126,000,000","5,256,000,000","4,710,000,000","5,644,000,000",NaN,NaN
Interest Income from Loans,"10,247,000,000","10,247,000,000","8,959,000,000","7,096,000,000","6,628,000,000","6,126,000,000","5,256,000,000","4,710,000,000","5,644,000,000",NaN,NaN
Interest Income from Securities,"4,468,000,000","4,468,000,000","4,020,000,000","3,116,000,000","2,749,000,000","3,052,000,000","3,115,000,000","2,816,000,000","1,481,000,000",NaN,NaN
Interest Income from Deposits,"877,000,000","877,000,000","819,000,000","621,000,000","371,000,000","466,000,000","577,000,000","460,000,000","496,000,000",NaN,NaN
"Interest Expense, Cost of Revenue","-5,967,000,000","-5,967,000,000","-4,843,000,000","-3,042,000,000","-2,443,000,000","-2,544,000,000","-2,627,000,000","-2,417,000,000","-2,336,000,000","-1,730,000,000","-1,381,000,000"
Interest Expense for Deposit,"-4,129,000,000","-4,129,000,000","-3,488,000,000","-2,180,000,000","-1,726,000,000","-1,940,000,000","-2,086,000,000","-1,926,000,000","-1,953,000,000",NaN,NaN


In [42]:
cash = pd.read_excel("cashflow.xls",  index_col=0)
cash

,TTM,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010
Name,,,,,,,,,,,
"Cash Flow from Operating Activities, Indirect","11,186,000,000","11,186,000,000","-1,283,000,000","-3,122,000,000","9,437,000,000","2,874,000,000","3,976,000,000","1,293,000,000","-6,908,000,000","-1,677,000,000","4,847,000,000"
"Net Cash Flow from Continuing Operating Activities, Indirect","11,186,000,000","11,186,000,000","-1,283,000,000","-3,122,000,000","9,437,000,000","2,874,000,000","3,976,000,000","1,293,000,000","-6,908,000,000","-1,677,000,000","4,847,000,000"
Cash Generated from Operating Activities,"11,821,000,000","11,821,000,000","-392,000,000","-2,413,000,000","10,246,000,000","3,604,000,000","4,709,000,000","1,855,000,000","-6,321,000,000","-1,166,000,000","3,369,000,000"
Income/Loss before Non-Cash Adjustment,"7,583,000,000","7,583,000,000","6,659,000,000","5,175,000,000","4,360,000,000","4,567,000,000","4,185,000,000","3,874,000,000","4,019,000,000","3,290,000,000",NaN
Total Adjustments for Non-Cash Items,"1,179,000,000","1,179,000,000","954,000,000","1,603,000,000","2,318,000,000","1,389,000,000","1,074,000,000","979,000,000","149,000,000","716,000,000","2,061,000,000"
...,...,...,...,...,...,...,...,...,...,...,...
Other Changes,0,0,"-3,000,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Cash and Cash Equivalents, Beginning of Period","14,221,000,000","14,221,000,000","18,693,000,000","20,132,000,000","12,078,000,000","11,851,000,000","10,949,000,000","10,993,000,000","18,891,000,000","25,112,000,000","19,281,000,000"
"Change in Cash As Reported, Supplemental","5,714,000,000","5,714,000,000","-4,469,000,000","-1,439,000,000","8,054,000,000","227,000,000","902,000,000","-44,000,000","-7,898,000,000","-6,221,000,000","5,831,000,000"


In [44]:
keyRatios = pd.read_csv('keyratios.csv', header=None,)
keyRatios

ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 12
